# Graphlab test notebook

Test whether it is possible to efficiently have a multigraph without duplicating values on the nodes.

In [15]:
import os
import graphlab as gl
import numpy as np
import scipy as sp

from collections import OrderedDict
import numpy.random

In [48]:
def generate_data(nb_users, nb_movies, max_rating=5):
    u = np.random.randint(1, nb_users + 1, nb_users)
    m = np.random.randint(1, nb_movies + 1, nb_movies)
    # unique pairs
    v = set(zip(u, m))
    data = {'user_id': list(), 'movie_id': list(), 'rating': list(), 'type': list()}
    for (u, m), r in zip(v, np.random.randint(1, max_rating + 1, nb_users)):
        data['user_id'].append(u)
        data['movie_id'].append(-m)  # negative id to distinguish between user and movie in graph
        data['rating'].append(r)
        data['type'].append('um')  # edge type user-movie
    return data

user_movie = gl.SFrame(generate_data(5, 5))
user_movie

movie_id,rating,type,user_id
-3,4,um,1
-1,4,um,5
-1,4,um,3
-2,5,um,5
-1,1,um,2


### Create vertices with respective types

In [49]:
m = gl.SFrame({'__id': user_movie['movie_id']})
m['ntype'] = 'movie'
u = gl.SFrame({'__id': user_movie['user_id']})
u['ntype'] = 'user'
v = m.append(u)
v

__id,ntype
-3,movie
-1,movie
-1,movie
-2,movie
-1,movie
1,user
5,user
3,user
5,user
2,user


In [50]:
g = gl.SGraph(vertices=v, edges=user_movie, src_field='user_id', dst_field='movie_id')

### Add movie to movie edges